In [ ]:
#| default_exp target_transforms

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# Target transforms
Transformations that can be applied to the target before computing the features and restored after computing the predictions.

In [ ]:
#| export
import abc
import copy
import reprlib
from typing import Iterable, List, Optional

import numpy as np
import pandas as pd
from sklearn.base import TransformerMixin, clone
from numba import njit
from utilsforecast.compat import DataFrame
from utilsforecast.target_transforms import (
    LocalBoxCox as BoxCox,
    LocalMinMaxScaler as MinMaxScaler,
    LocalRobustScaler as RobustScaler,
    LocalStandardScaler as StandardScaler,
    _common_scaler_inverse_transform,
    _transform,
)

from mlforecast.grouped_array import GroupedArray, _apply_difference
from mlforecast.utils import _ensure_shallow_copy, _ShortSeriesException

In [ ]:
import pandas as pd
from fastcore.test import test_fail
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PowerTransformer

from mlforecast import MLForecast
from mlforecast.utils import generate_daily_series

In [ ]:
#| export
class BaseTargetTransform(abc.ABC):
    """Base class used for target transformations."""
    def set_column_names(self, id_col: str, time_col: str, target_col: str):
        self.id_col = id_col
        self.time_col = time_col
        self.target_col = target_col

    @abc.abstractmethod
    def fit_transform(self, df: DataFrame) -> DataFrame:
        raise NotImplementedError
        
    @abc.abstractmethod
    def inverse_transform(self, df: DataFrame) -> DataFrame:
        raise NotImplementedError

In [ ]:
#| export
class BaseGroupedArrayTargetTransform(abc.ABC):
    """Base class used for target transformations that operate on grouped arrays."""
    idxs: Optional[np.ndarray] = None

    @abc.abstractmethod
    def fit_transform(self, ga: GroupedArray) -> GroupedArray:
        raise NotImplementedError
        
    @abc.abstractmethod
    def inverse_transform(self, ga: GroupedArray) -> GroupedArray:
        raise NotImplementedError

In [ ]:
#| export
class Differences(BaseGroupedArrayTargetTransform):
    """Subtracts previous values of the serie. Can be used to remove trend or seasonalities."""
    store_fitted = False
    
    def __init__(self, differences: Iterable[int]):
        self.differences = list(differences)

    def fit_transform(self, ga: GroupedArray) -> GroupedArray:
        ga = copy.copy(ga)
        self.fitted_: List[GroupedArray] = []
        original_sizes = np.diff(ga.indptr)
        total_diffs = sum(self.differences)
        small_series = original_sizes < total_diffs
        if small_series.any():
            raise _ShortSeriesException(np.arange(ga.ngroups)[small_series])
        self.original_values_ = []
        n_series = len(ga.indptr) - 1
        for d in self.differences:
            if self.store_fitted:
                # these are saved in order to be able to perform a correct
                # inverse transform when trying to retrieve the fitted values.
                self.fitted_.append(copy.copy(ga))
            new_data = np.empty_like(ga.data, shape=n_series * d)
            new_indptr = d * np.arange(n_series + 1, dtype=np.int32)
            _apply_difference(ga.data, ga.indptr, new_data, new_indptr, d)
            self.original_values_.append(GroupedArray(new_data, new_indptr))
        return ga

    def inverse_transform(self, ga: GroupedArray) -> GroupedArray:
        ga = copy.copy(ga)
        for d, orig_vals_ga in zip(reversed(self.differences), reversed(self.original_values_)):
            if self.idxs is not None:
                orig_vals_ga = orig_vals_ga.take(self.idxs)
            orig_vals_ga.restore_difference(ga.data, d)
        return ga

    def inverse_transform_fitted(self, ga: GroupedArray) -> pd.DataFrame:
        ga = copy.copy(ga)
        for d, orig_vals_ga in zip(reversed(self.differences), reversed(self.fitted_)):
            orig_vals_ga.restore_fitted_difference(ga.data, ga.indptr, d)
        return ga

In [ ]:
series = generate_daily_series(10, min_length=50, max_length=100)

In [ ]:
diffs = Differences([1, 2, 5])
ga = GroupedArray.from_sorted_df(series, 'unique_id', 'ds', 'y')

# differences are applied correctly
transformed = diffs.fit_transform(ga)
assert diffs.fitted_ == []
expected = series.copy()
for d in diffs.differences:
    expected['y'] -= expected.groupby('unique_id')['y'].shift(d)
np.testing.assert_allclose(transformed.data, expected['y'].values)

# fitted differences are restored correctly
diffs.store_fitted = True
transformed = diffs.fit_transform(ga)
keep_mask = ~np.isnan(transformed.data)
restored = diffs.inverse_transform_fitted(transformed)
np.testing.assert_allclose(ga.data[keep_mask], restored.data[keep_mask])
restored_subs = diffs.inverse_transform_fitted(transformed.take_from_groups(slice(8, None)))
np.testing.assert_allclose(ga.data[keep_mask], restored_subs.data)

# short series
ga = GroupedArray(np.arange(20), np.array([0, 2, 20]))
test_fail(lambda: diffs.fit_transform(ga), contains="[0]")

In [ ]:
#| exporti
class BaseLocalScaler(BaseGroupedArrayTargetTransform):
    """Standardizes each serie by subtracting its mean and dividing by its standard deviation."""
    scaler_factory: type
    
    def fit_transform(self, ga: GroupedArray) -> GroupedArray:
        self.scaler_ = self.scaler_factory()
        transformed = self.scaler_.fit_transform(ga)
        return GroupedArray(transformed, ga.indptr)

    def inverse_transform(self, ga: GroupedArray) -> GroupedArray:
        stats = self.scaler_.stats_
        if self.idxs is not None:
            stats = stats[self.idxs]
        transformed = _transform(ga.data, ga.indptr, stats, _common_scaler_inverse_transform)
        return GroupedArray(transformed, ga.indptr)

In [ ]:
def test_scaler(sc, series):
    ga = GroupedArray.from_sorted_df(series, 'unique_id', 'ds', 'y')
    transformed = sc.fit_transform(ga)
    np.testing.assert_allclose(
        sc.inverse_transform(transformed).data,
        ga.data,
    )
    
    def filter_df(df):
        return (
            df[df['unique_id'].isin(['id_0', 'id_7'])]
            .groupby('unique_id', observed=True)
            .head(10)
        )
    
    idxs = [0, 7]
    subset = ga.take(idxs)
    transformed_subset = transformed.take(idxs)
    sc.idxs = idxs
    np.testing.assert_allclose(
        sc.inverse_transform(transformed_subset).data,
        subset.data,
    )    

In [ ]:
#| export
class LocalStandardScaler(BaseLocalScaler):
    """Standardizes each serie by subtracting its mean and dividing by its standard deviation."""
    scaler_factory = StandardScaler

In [ ]:
test_scaler(LocalStandardScaler(), series)

In [ ]:
#| export
class LocalMinMaxScaler(BaseLocalScaler):
    """Scales each serie to be in the [0, 1] interval."""
    scaler_factory = MinMaxScaler

In [ ]:
test_scaler(LocalMinMaxScaler(), series)

In [ ]:
#| export
class LocalRobustScaler(BaseLocalScaler):
    """Scaler robust to outliers.

    Parameters
    ----------
    scale : str (default='iqr')
        Statistic to use for scaling. Can be either 'iqr' (Inter Quartile Range) or 'mad' (Median Asbolute Deviation)
    """

    def __init__(self, scale: str):
        self.scaler_factory = lambda: RobustScaler(scale)

In [ ]:
test_scaler(LocalRobustScaler(scale='iqr'), series)

In [ ]:
test_scaler(LocalRobustScaler(scale='mad'), series)

In [ ]:
#| export
class GlobalSklearnTransformer(BaseTargetTransform):
    """Applies the same scikit-learn transformer to all series."""    
    def __init__(self, transformer: TransformerMixin):
        self.transformer = transformer

    def fit_transform(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy(deep=False)
        self.transformer_ = clone(self.transformer)
        df[self.target_col] = self.transformer_.fit_transform(df[[self.target_col]].values)
        return df

    def inverse_transform(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy(deep=False)
        cols_to_transform = df.columns.drop([self.id_col, self.time_col])
        for col in cols_to_transform:
            df[col] = self.transformer_.inverse_transform(df[[col]].values)
        return df

In [ ]:
# need this import in order for isinstance to work
from mlforecast.target_transforms import Differences as ExportedDifferences

In [ ]:
sk_boxcox = PowerTransformer(method='box-cox', standardize=False)
boxcox_global = GlobalSklearnTransformer(sk_boxcox)
single_difference = ExportedDifferences([1])
series = generate_daily_series(10)
fcst = MLForecast(
    models=[LinearRegression()],
    freq='D',
    lags=[1, 2],
    target_transforms=[boxcox_global, single_difference]
)
prep = fcst.preprocess(series, dropna=False)
expected = (
    pd.Series(
        sk_boxcox.fit_transform(series[['y']])[:, 0], index=series['unique_id']
    ).groupby('unique_id')
    .diff()
    .values
)
np.testing.assert_allclose(prep['y'].values, expected)